In [175]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
data=pd.read_csv('Resources/aviation_renamed.csv')
data.head()

,Unnamed: 0,Date,Country,City_State,Make,Model,Injury_Severity,Aircraft_Damage,Latitude,Longitude
0,0,12/31/2019,United States,"Elk, CA",Cessna,175,Non-Fatal,Substantial,39.128611,-123.715833
1,1,12/31/2019,United States,"OLATHE, KS",Mooney,M20S,Fatal(2),Destroyed,38.846111,-94.736111
2,2,12/31/2019,United States,"Fairbanks, AK",Cessna,170,Non-Fatal,Substantial,64.666945,-148.133334
3,5,12/28/2019,United States,"LAFAYETTE, LA",Piper,PA 31T,Fatal(5),Destroyed,30.176111,-92.007500
4,6,12/25/2019,United States,"Headland, AL",Bell,407,Fatal(1),Substantial,31.364167,-85.312500


In [182]:
planes=pd.DataFrame({'Make and Model':[],
                     'Description':[],
                     'URL':[]
                     })

In [90]:
data['Make']=data['Make'].str.lower()
data['Make and Model']=data['Make']+' '+data['Model']
makemodel=data['Make and Model'].value_counts()
makemodel=pd.DataFrame(makemodel)
makemodel=makemodel.reset_index(drop=False).rename(columns={'index':'Make And Model', 'Make and Model': 'count'})
makemodel

,Make And Model,count
0,cessna 172,585
1,cessna 152,381
2,cessna 172S,324
3,cessna 172N,316
4,cessna 180,229
...,...,...
5074,ercoupe (eng & research corp.) 415-CD,1
5075,cessna C-150H,1
5076,eurocopter AS 350 B2 ECUREUIL,1
5077,beech BE-V35B,1


In [183]:

for i in range(len(data)):
    base_url='https://en.wikipedia.org/wiki/Special:Search?search='
    mm2=makemodel['Make And Model'][i]
    mm=makemodel['Make And Model'][i].replace(' ','+')
    url2=mm+'&go=Go&ns0=1'
    url=base_url+url2
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    try:


        articles = soup.find('p')
        text=articles.text.strip()
        script=soup.body.find_all('script', type="application/ld+json")[0]
        wiki_link=json.loads(script.string)['url']
        plane=pd.DataFrame({'Make and Model':[makemodel['Make And Model'][i]],
                            'Description':[text],
                            'URL':[wiki_link]
                            })
        print(f'Plane found:{mm2}')
    except:
        try:
            article2 = soup.find('div', class_='mw-search-result-heading')
            click=article2.text[:-1].replace(' ','_')
            browser.click_link_by_href('/wiki/'+click)
            time.sleep(2)
        except:
            print('No clickable link!')
        try:
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            article2 = soup.find('p')
            text=article2.text.strip()
            script=soup.body.find_all('script', type="application/ld+json")[0]
            wiki_link=json.loads(script.string)['url']
            plane=pd.DataFrame({'Make and Model':[makemodel['Make And Model'][i]],
                                'Description':[text],
                                'URL':[wiki_link]
                                })
            print(f'Plane found:{mm2}')
        except:
            print('did not work')
            


    planes=planes.append(plane)
    if i==9:
        break
planes

Plane found:cessna 172
Plane found:cessna 152
Plane found:cessna 172S
Plane found:cessna 172N
Plane found:cessna 180
Plane found:cessna 172M
Plane found:cessna 182
Plane found:piper PA-28-140
Plane found:piper PA-18-150
Plane found:cessna 172P


,Make and Model,Description,URL
0,cessna 172,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 152,"The Cessna 152 is an American two-seat, fixed-...",https://en.wikipedia.org/wiki/Cessna_152
0,cessna 172S,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 172N,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 180,"The Cessna 180 is a four- or six-seat, fixed c...",https://en.wikipedia.org/wiki/Cessna_180
0,cessna 172M,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
0,cessna 182,The Cessna 182 Skylane is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_182_Skylane
0,piper PA-28-140,The Piper PA-28 Cherokee is a family of two-se...,https://en.wikipedia.org/wiki/Piper_PA-28_Cher...
0,piper PA-18-150,"The Piper PA-18 Super Cub is a two-seat, singl...",https://en.wikipedia.org/wiki/Piper_PA-18_Supe...
0,cessna 172P,The Cessna 172 Skyhawk is an American four-sea...,https://en.wikipedia.org/wiki/Cessna_172
